In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve
)

In [2]:
data = pd.read_csv("bank.csv", sep=";")

Separating Features & Target

In [3]:
X = data.drop("y", axis=1)
y = data["y"].map({"yes": 1, "no": 0})  # binary encoding

Identifying Categorical & Numerical Columns

In [4]:
categorical_cols = X.select_dtypes(include=["object"]).columns
numerical_cols = X.select_dtypes(exclude=["object"]).columns

Encoding

In [5]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numerical_cols)
    ]
)

Train–Test Split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

Building Logistic Regression Pipeline

In [7]:
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000))
])

model.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome'],
      dtype='object')),
                                                 ('num', 'passthrough',
                                                  Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous'], dtype='object'))])),
                ('classifier', LogisticRegression(max_iter=1000))])

Predictions & Probabilities

In [8]:
y_prob = model.predict_proba(X_test)[:, 1]

Evaluation at Threshold = 0.5

In [9]:
def evaluate_model(y_true, y_prob, threshold):
    y_pred = (y_prob >= threshold).astype(int)

    cm = confusion_matrix(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)  # Sensitivity
    f1 = f1_score(y_true, y_pred)

    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp)

    return cm, precision, recall, f1, specificity

In [10]:
cm_05, p_05, r_05, f1_05, spec_05 = evaluate_model(y_test, y_prob, 0.5)

print("Threshold = 0.5")
print("Confusion Matrix:\n", cm_05)
print(f"Precision: {p_05:.4f}")
print(f"Recall (Sensitivity): {r_05:.4f}")
print(f"Specificity: {spec_05:.4f}")
print(f"F1-score: {f1_05:.4f}")

Threshold = 0.5
Confusion Matrix:
 [[969  32]
 [ 92  38]]
Precision: 0.5429
Recall (Sensitivity): 0.2923
Specificity: 0.9680
F1-score: 0.3800


ROC–AUC Score

In [12]:
roc_auc = roc_auc_score(y_test, y_prob)
print(f"ROC-AUC Score: {roc_auc:.4f}")

ROC-AUC Score: 0.8874


In [13]:
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
youden_j = tpr - fpr
best_threshold = thresholds[np.argmax(youden_j)]

print(f"Optimized Threshold: {best_threshold:.4f}")

Optimized Threshold: 0.0857


In [14]:
cm_opt, p_opt, r_opt, f1_opt, spec_opt = evaluate_model(
    y_test, y_prob, best_threshold
)

print("\nOptimized Threshold Results")
print("Confusion Matrix:\n", cm_opt)
print(f"Precision: {p_opt:.4f}")
print(f"Recall (Sensitivity): {r_opt:.4f}")
print(f"Specificity: {spec_opt:.4f}")
print(f"F1-score: {f1_opt:.4f}")


Optimized Threshold Results
Confusion Matrix:
 [[764 237]
 [ 18 112]]
Precision: 0.3209
Recall (Sensitivity): 0.8615
Specificity: 0.7632
F1-score: 0.4676


In [15]:
output = pd.DataFrame({
    "RecordId": y_test.index,
    "Probability(yes)": y_prob,
    "PredictedLabel": (y_prob >= best_threshold).astype(int)
})

output.to_csv("probabilities.csv", index=False)

### Why is the ROC curve useful? ###

The ROC curve evaluates a classification model across all decision thresholds by plotting True Positive Rate (Recall) against False Positive Rate. It shows how well the model can distinguish between classes independent of any specific threshold. The ROC-AUC score summarizes this ability into a single value, making it especially useful for imbalanced datasets.

### What changes when the threshold changes? (Precision–Recall trade-off) ###

Changing the classification threshold alters the balance between precision and recall.

Lowering the threshold increases recall but decreases precision due to more false positives.

Increasing the threshold improves precision but reduces recall due to more false negatives.

Thus, threshold selection controls the trade-off between capturing more positives and avoiding false alarms, and should be chosen based on business cost considerations.